In [29]:
import yfinance as yf
import numpy as np
import pandas as pd

# 1. Download Apple (AAPL) stock data, auto-adjusted for splits/dividends
df = yf.download('AAPL', start='2020-01-01', auto_adjust=True)

# 2. If MultiIndex (sometimes happens with yfinance), drop the ticker level
if isinstance(df.columns, pd.MultiIndex):
    df.columns = df.columns.droplevel(1)

print(df.columns)
# Now df has columns like: ['Open','High','Low','Close','Volume']

# 3. Compute daily log returns
df['ret'] = np.log(df['Close']).diff()

# 4. Compute features
df['mom_5'] = df['ret'].rolling(5).mean()   # 5-day momentum (trend indicator)
df['vol_5'] = df['ret'].rolling(5).std()    # 5-day volatility (risk indicator)

# 5. Create trading signal labels: Buy if tomorrow’s return > 0, else Sell
df['label'] = np.where(df['ret'].shift(-1) > 0, "Buy", "Sell")

# 6. Show last 10 rows neatly formatted
print(df[['Close', 'ret', 'mom_5', 'vol_5', 'label']].tail(10).to_string())

[*********************100%***********************]  1 of 1 completed

Index(['Close', 'High', 'Low', 'Open', 'Volume'], dtype='object', name='Price')
Price            Close       ret     mom_5     vol_5 label
Date                                                      
2025-08-08  229.090012  0.041485  0.025020  0.022712  Sell
2025-08-11  227.179993 -0.008372  0.022390  0.026145   Buy
2025-08-12  229.649994  0.010814  0.024976  0.023634   Buy
2025-08-13  233.330002  0.015897  0.018225  0.019234  Sell
2025-08-14  232.779999 -0.002360  0.011493  0.019403  Sell
2025-08-15  231.589996 -0.005125  0.002171  0.010583  Sell
2025-08-18  230.889999 -0.003027  0.003240  0.009463  Sell
2025-08-19  230.559998 -0.001430  0.000791  0.008553  Sell
2025-08-20  226.009995 -0.019932 -0.006375  0.007700  Sell
2025-08-21  224.899994 -0.004923 -0.006888  0.007447  Sell


## Read results:

Close → adjusted stock price.

ret → daily log return.

mom_5 → average of last 5 daily returns (positive = trend up).

vol_5 → riskiness over last 5 days (higher = more volatile).

label → your trading signal:

**"Buy"** if next day’s return was > 0

**"Sell"** if next day’s return was ≤ 0